In [ ]:
import numpy as np
import scipy.io
import os
import glob
import json

In [ ]:
#Global variables

sampling_rate=512;
bin_size=10;

In [ ]:
from scipy.signal import hilbert

def amp_en(alldata, bin_size=bin_size):
    number_of_datapoints, number_of_channels = alldata.shape
    
    # Filtering all the channels
    Aamplitude = np.empty((number_of_datapoints, number_of_channels))
    for i in range(number_of_channels):
        Aamplitude[:, i] = np.abs(hilbert(alldata[:, i]))
    
    en = np.empty(number_of_datapoints)
    
    # Calculating Entropies across channels
    for t in range(number_of_datapoints):
        hist, bin_edges = np.histogram(Aamplitude[t, :], bins=np.arange(min(Aamplitude[t, :]), max(Aamplitude[t, :]) + bin_size, bin_size))
        pmf = hist / np.sum(hist)
        pmf = pmf[pmf > 0]
        en[t] = -np.sum(pmf * np.log2(pmf))
    
    return en

In [ ]:
output_events={'fileID':[],'pat_id':[],'sez_id':[],'sez_len':[],'elec_no':[],'aa':[]}

#Modify the Path here acording to the download location
data_path="/home/sapta/Documents/"

for patid in range(1,17):
    mat_files = glob.glob(os.path.join(data_path+"ID"+str(patid), '*.mat'))
    no_of_seizures = len(mat_files)
    #print("No of seizures: ", no_of_seizures)
    #Slength=[];no_of_electrodes=[];

    for i in range(1,no_of_seizures+1):
        filename = os.path.join(data_path, "ID"+str(patid)+"/Sz"+str(i)+".mat")
        mat=scipy.io.loadmat(filename)
        data=np.array(mat.get('EEG'))

        #Sampling rate is 512Hz and each seizure preceded and succeeded by a 3 mins long segment
        sez_length=(np.shape(data)[0]/sampling_rate) - (2*3*60) #in seconds
        no_elec=np.shape(data)[1]
        
        # Slength.append(sez_length) #in seconds
        # no_of_electrodes.append(no_elec)
        
        output_events['fileID'].append("p"+str(patid)+"s"+str(i))#Recorded in seconds
        output_events['pat_id'].append("ID"+str(patid))
        output_events['sez_id'].append(i)#Recorded in seconds
        output_events['sez_len'].append(sez_length)#Recorded in seconds
        output_events['elec_no'].append(no_elec)
        output_events['aa'].append(amp_en(data))

In [ ]:
import pandas as pd
out=pd.DataFrame.from_dict(output_events)
out.to_json("../../Code_4/all_unfiltered_data_Swiss-Short.json", orient='records')

In [ ]:
# result = out.to_json(orient="records")
# parsed = json.loads(result)
# with open("../../Code_3/all_data_Swiss-Short.json", "w") as write_file:
#         json.dump(parsed, write_file, indent=4)